In [108]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.models import Model

import os
import tempfile
import math

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [83]:
model = tf.keras.models.load_model('../input/no-obs-2/no_obs')

In [84]:
img_height, img_width = 200, 200
data_dir = '../input/no-obs-diffs/no_obs_diffs'
batch_size = 32

In [85]:
datagen = ImageDataGenerator(
    validation_split=0.3,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary',
    subset='validation')

Found 5752 images belonging to 2 classes.


In [86]:
n_val_images = val_generator.samples

In [87]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('No Obstacles Detected (True Negatives): ', cm[0][0])
  print('No Obstacles Incorrectly Detected (False Positives): ', cm[0][1])
  print('Obstacles Missed (False Negatives): ', cm[1][0])
  print('Obstacles Detected (True Positives): ', cm[1][1])
  print('Total Obstacles: ', np.sum(cm[1]))

In [88]:
metrics = model.evaluate_generator(
    val_generator,
    verbose=1)

180/180 [==============================] - 44s 247ms/step - loss: 2.1637 - tp: 2593.0000 - fp: 155.0000 - tn: 2891.0000 - fn: 113.0000 - accuracy: 0.9534 - precision: 0.9436 - recall: 0.9582 - auc: 0.9888


In [89]:
for name, value in zip(model.metrics_names, metrics):
  print(name, ': ', value)
print()


loss :  2.163747787475586
tp :  2593.0
fp :  155.0
tn :  2891.0
fn :  113.0
accuracy :  0.9534075260162354
precision :  0.9435953497886658
recall :  0.9582409262657166
auc :  0.9888389110565186



In [90]:
num_neg = 0

for i, f in enumerate(val_generator.filenames):
    if f[:11] != 'no_obstacle':
        num_neg = i
        break
        
print('num of negatives:', num_neg)

num of negatives: 3046


In [91]:
num_pos = n_val_images - num_neg
        
print('num of positives:', num_pos)

num of positives: 2706


In [111]:
labels = np.array([0]*num_neg + [1]*num_pos)
n_batches = math.ceil(n_val_images/batch_size)

In [93]:
predictions = model.predict_generator(val_generator, n_batches, verbose=1)

167/167 [==============================] - 41s 248ms/step


In [94]:
val_generator.class_indices

{'no_obstacle': 0, 'obstacle': 1}

In [95]:
neg = predictions[:num_neg]
fp = []

for i,p in enumerate(neg):
    if p >= 0.5:
        fp.append(i)

In [96]:
len(fp)

158

In [97]:
fp

[16,
 28,
 41,
 54,
 67,
 80,
 90,
 94,
 108,
 122,
 136,
 149,
 162,
 175,
 188,
 201,
 214,
 227,
 240,
 253,
 266,
 275,
 279,
 291,
 304,
 317,
 330,
 343,
 356,
 369,
 382,
 395,
 408,
 421,
 434,
 447,
 460,
 473,
 486,
 495,
 499,
 512,
 525,
 537,
 549,
 558,
 571,
 572,
 585,
 586,
 1466,
 1472,
 1481,
 1488,
 1499,
 1511,
 1532,
 1541,
 1554,
 1565,
 1575,
 1589,
 1603,
 1778,
 1779,
 1782,
 1807,
 1814,
 1815,
 1819,
 1823,
 1827,
 1831,
 1835,
 1847,
 1851,
 1855,
 1863,
 1867,
 1871,
 1882,
 1883,
 1886,
 1887,
 1890,
 1891,
 1894,
 1898,
 1899,
 1907,
 1910,
 1911,
 1919,
 1923,
 1927,
 1930,
 1931,
 1934,
 1935,
 1939,
 1943,
 1947,
 1951,
 1955,
 1958,
 1959,
 1963,
 1967,
 1970,
 1971,
 1974,
 1975,
 1979,
 1983,
 1987,
 1991,
 1994,
 1995,
 2002,
 2007,
 2011,
 2015,
 2019,
 2023,
 2026,
 2027,
 2031,
 2035,
 2039,
 2046,
 2047,
 2051,
 2055,
 2059,
 2063,
 2067,
 2075,
 2078,
 2083,
 2086,
 2087,
 2091,
 2095,
 2099,
 2103,
 2110,
 2111,
 2123,
 2130,
 2131,
 2134,
 

In [98]:
val_generator.filenames[8]

'no_obstacle/datalight_test1mp4_f0_o2.png'

In [99]:
pos = predictions[num_neg:]
fn = []

for i,p in enumerate(pos):
    if p < 0.5:
        fn.append(i + num_neg)

In [100]:
pos

array([[0.9998329 ],
       [0.934226  ],
       [0.99954236],
       ...,
       [0.20222646],
       [0.5143779 ],
       [0.65265465]], dtype=float32)

In [101]:
len(fn)

88

In [102]:
fn

[3185,
 3424,
 3427,
 3430,
 3433,
 3469,
 3471,
 3473,
 3475,
 3477,
 3482,
 3508,
 3510,
 3813,
 3814,
 3815,
 3817,
 3819,
 3821,
 3824,
 3827,
 3842,
 3855,
 3857,
 3859,
 3861,
 3863,
 3865,
 3867,
 3869,
 3871,
 3873,
 3875,
 3877,
 3984,
 3986,
 4155,
 4159,
 4161,
 4163,
 4165,
 4168,
 4170,
 4172,
 4174,
 4176,
 5185,
 5187,
 5188,
 5189,
 5191,
 5192,
 5193,
 5195,
 5196,
 5201,
 5229,
 5311,
 5312,
 5313,
 5314,
 5315,
 5316,
 5317,
 5318,
 5319,
 5320,
 5321,
 5322,
 5323,
 5324,
 5325,
 5326,
 5327,
 5328,
 5329,
 5330,
 5331,
 5332,
 5333,
 5334,
 5335,
 5336,
 5337,
 5338,
 5339,
 5340,
 5341]

In [103]:
val_generator.filenames[2792]

'no_obstacle/datalight_test_japan_mor3mp4_f1194_o54.png'

In [104]:
plot_cm(labels, predictions) # Default: threshold = 0.5
plot_cm(labels, predictions, p=0.25)
plot_cm(labels, predictions, p=0.75)

ValueError: Found input variables with inconsistent numbers of samples: [5752, 5344]